# 텍스트-이미지 변환

확산 모델을 생각할 때 일반적으로 가장 먼저 떠오르는 것 중 하나가 텍스트-이미지 변환입니다. 텍스트-이미지 변환은 텍스트 설명(예: "정글 속 우주비행사, 차가운 색상 팔레트, 차분한 색상, 디테일, 8k")에서 이미지를 생성하며, 이를 *프롬프트*라고도 합니다.

매우 높은 수준에서 확산 모델은 프롬프트와 일부 임의의 초기 노이즈를 취하고 반복적으로 노이즈를 제거하여 이미지를 구성합니다. *노이즈 제거* 프로세스는 프롬프트에 의해 안내되며, 미리 결정된 시간 단계 수 후에 노이즈 제거 프로세스가 종료되면 이미지 표현이 이미지로 디코딩됩니다.

<Tip>

잠재 확산 모델의 작동 방식에 대해 자세히 알아보려면 [Stable Diffusion은 어떻게 작동합니까?](https://huggingface.co/blog/stable_diffusion#how-does-stable-diffusion-work) 블로그 게시물을 읽어보세요.

</Tip>

🤗 Diffusers에서 프롬프트에서 이미지를 생성하는 두 단계는 다음과 같습니다.

1. 체크포인트를 기반으로 적절한 파이프라인 클래스를 자동으로 감지하는 [AutoPipelineForText2Image](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForText2Image) 클래스로 체크포인트를 로드합니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained(
	"stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16"
).to("cuda")

2. 이미지를 생성하기 위해 파이프라인에 프롬프트를 전달합니다.

In [ ]:
image = pipeline(
	"stained glass of darth vader, backlight, centered composition, masterpiece, photorealistic, 8k"
).images[0]
image

<div class="flex justify-center">
	<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/text2img-vader.png"/>
</div>

## 인기 모델

가장 일반적인 텍스트-이미지 모델은 [Stable Diffusion v1.5](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5), [Stable Diffusion XL (SDXL)](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0) 및 [Kandinsky 2.2](https://huggingface.co/kandinsky-community/kandinsky-2-2-decoder)입니다. 이미지 생성을 보다 직접적으로 제어하기 위해 텍스트-이미지 모델과 함께 사용할 수 있는 ControlNet 모델 또는 어댑터도 있습니다. 각 모델의 결과는 아키텍처 및 학습 프로세스로 인해 약간 다르지만 선택한 모델에 관계없이 사용법은 거의 동일합니다. 각 모델에 대해 동일한 프롬프트를 사용하고 결과를 비교해 보겠습니다.

### 스테이블 디퓨전 v1.5

[Stable Diffusion v1.5](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5)는 [Stable Diffusion v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4)에서 초기화된 잠재 확산 모델이며, LAION-Aesthetics V2 데이터셋의 512x512 이미지에서 595K 단계 동안 미세 조정되었습니다. 이 모델은 다음과 같이 사용할 수 있습니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained(
	"stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16"
).to("cuda")
generator = torch.Generator("cuda").manual_seed(31)
image = pipeline("Astronaut in a jungle, cold color palette, muted colors, detailed, 8k", generator=generator).images[0]
image

### 스테이블 디퓨전 XL

SDXL은 이전 Stable Diffusion 모델보다 훨씬 큰 버전이며, 이미지에 훨씬 더 많은 세부 정보를 추가하는 2단계 모델 프로세스를 포함합니다. 또한 중앙에 위치한 피사체의 고품질 이미지를 생성하기 위한 몇 가지 추가적인 *미세 조건화*도 포함합니다. 사용 방법에 대해 자세히 알아보려면 보다 포괄적인 [SDXL](https://huggingface.co/docs/diffusers/main/en/using-diffusers/sdxl) 가이드를 참조하세요. 일반적으로 SDXL은 다음과 같이 사용할 수 있습니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16"
).to("cuda")
generator = torch.Generator("cuda").manual_seed(31)
image = pipeline("Astronaut in a jungle, cold color palette, muted colors, detailed, 8k", generator=generator).images[0]
image

### 칸딘스키 2.2

Kandinsky 모델은 Stable Diffusion 모델과 약간 다릅니다. 왜냐하면 이미지 사전 모델을 사용하여 확산 모델에서 텍스트와 이미지를 더 잘 정렬하는 데 사용되는 임베딩을 생성하기 때문입니다.

Kandinsky 2.2를 사용하는 가장 쉬운 방법은 다음과 같습니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained(
	"kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16
).to("cuda")
generator = torch.Generator("cuda").manual_seed(31)
image = pipeline("Astronaut in a jungle, cold color palette, muted colors, detailed, 8k", generator=generator).images[0]
image

### ControlNet

ControlNet 모델은 [Stable Diffusion v1.5](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5)와 같은 텍스트-이미지 모델 위에 미세 조정된 보조 모델 또는 어댑터입니다. ControlNet 모델을 텍스트-이미지 모델과 함께 사용하면 이미지를 생성하는 방법을 보다 명시적으로 제어할 수 있는 다양한 옵션을 제공합니다. ControlNet을 사용하면 모델에 추가 조건 입력 이미지를 추가합니다. 예를 들어, 인간 자세 이미지(일반적으로 골격으로 연결된 여러 키포인트로 표시됨)를 조건 입력으로 제공하면 모델은 이미지의 자세를 따르는 이미지를 생성합니다. 다른 조건 입력 및 사용 방법에 대해 자세히 알아보려면 보다 심층적인 [ControlNet](https://huggingface.co/docs/diffusers/main/en/using-diffusers/controlnet) 가이드를 확인하세요.

이 예에서는 인간 자세 추정 이미지로 ControlNet을 조건화해 보겠습니다. 인간 자세 추정에 사전 학습된 ControlNet 모델을 로드합니다.

In [ ]:
from diffusers import ControlNetModel, AutoPipelineForText2Image
from diffusers.utils import load_image
import torch

controlnet = ControlNetModel.from_pretrained(
	"lllyasviel/control_v11p_sd15_openpose", torch_dtype=torch.float16, variant="fp16"
).to("cuda")
pose_image = load_image("https://huggingface.co/lllyasviel/control_v11p_sd15_openpose/resolve/main/images/control.png")

`controlnet`을 [AutoPipelineForText2Image](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForText2Image)에 전달하고 프롬프트와 자세 추정 이미지를 제공합니다.

In [ ]:
pipeline = AutoPipelineForText2Image.from_pretrained(
	"stable-diffusion-v1-5/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16, variant="fp16"
).to("cuda")
generator = torch.Generator("cuda").manual_seed(31)
image = pipeline("Astronaut in a jungle, cold color palette, muted colors, detailed, 8k", image=pose_image, generator=generator).images[0]
image

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/text2img-1.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">스테이블 디퓨전 v1.5</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-text2img.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">스테이블 디퓨전 XL</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/text2img-2.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">칸딘스키 2.2</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/text2img-3.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">ControlNet (자세 조건화)</figcaption>
  </div>
</div>

## 파이프라인 매개변수 구성

이미지 생성 방식에 영향을 미치는 파이프라인에서 구성할 수 있는 여러 매개변수가 있습니다. 이미지의 출력 크기를 변경하고, 이미지 품질을 향상시키기 위해 부정적인 프롬프트를 지정하는 등 다양한 작업을 수행할 수 있습니다. 이 섹션에서는 이러한 매개변수를 사용하는 방법에 대해 자세히 설명합니다.

### 높이와 너비

`height` 및 `width` 매개변수는 생성된 이미지의 높이와 너비(픽셀 단위)를 제어합니다. 기본적으로 Stable Diffusion v1.5 모델은 512x512 이미지를 출력하지만 8의 배수인 모든 크기로 변경할 수 있습니다. 예를 들어 직사각형 이미지를 만들려면 다음을 수행합니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained(
	"stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16"
).to("cuda")
image = pipeline(
	"Astronaut in a jungle, cold color palette, muted colors, detailed, 8k", height=768, width=512
).images[0]
image

<div class="flex justify-center">
	<img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/text2img-hw.png"/>
</div>

<Tip warning={true}>

다른 모델은 학습 데이터셋의 이미지 크기에 따라 기본 이미지 크기가 다를 수 있습니다. 예를 들어 SDXL의 기본 이미지 크기는 1024x1024이며 낮은 `height` 및 `width` 값을 사용하면 이미지 품질이 저하될 수 있습니다. 먼저 모델의 API 참조를 확인하세요!

</Tip>

### 안내 척도

`guidance_scale` 매개변수는 프롬프트가 이미지 생성에 얼마나 영향을 미치는지에 영향을 줍니다. 값이 낮을수록 모델은 프롬프트와 느슨하게 관련된 이미지를 생성하는 "창의성"을 갖게 됩니다. `guidance_scale` 값이 높을수록 모델은 프롬프트를 더 면밀히 따르도록 하며, 이 값이 너무 높으면 생성된 이미지에 일부 아티팩트가 나타날 수 있습니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained(
	"stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16
).to("cuda")
image = pipeline(
	"Astronaut in a jungle, cold color palette, muted colors, detailed, 8k", guidance_scale=3.5
).images[0]
image

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/text2img-guidance-scale-2.5.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">guidance_scale = 2.5</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/text2img-guidance-scale-7.5.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">guidance_scale = 7.5</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/text2img-guidance-scale-10.5.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">guidance_scale = 10.5</figcaption>
  </div>
</div>

### 부정적 프롬프트

프롬프트가 생성을 안내하는 것처럼 *부정적 프롬프트*는 모델이 생성하지 않기를 바라는 것에서 모델을 멀리하도록 유도합니다. 이는 "저해상도" 또는 "나쁜 디테일"과 같은 좋지 않거나 나쁜 이미지 특징을 제거하여 전반적인 이미지 품질을 향상시키는 데 일반적으로 사용됩니다. 부정적 프롬프트를 사용하여 이미지의 내용과 스타일을 제거하거나 수정할 수도 있습니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained(
	"stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16
).to("cuda")
image = pipeline(
	prompt="Astronaut in a jungle, cold color palette, muted colors, detailed, 8k",
	negative_prompt="ugly, deformed, disfigured, poor details, bad anatomy",
).images[0]
image

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/text2img-neg-prompt-1.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">negative_prompt = "ugly, deformed, disfigured, poor details, bad anatomy"</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/text2img-neg-prompt-2.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">negative_prompt = "astronaut"</figcaption>
  </div>
</div>

### 생성기

[`torch.Generator`](https://pytorch.org/docs/stable/generated/torch.Generator.html#generator) 객체는 수동 시드를 설정하여 파이프라인에서 재현성을 가능하게 합니다. `Generator`를 사용하여 이미지 배치를 생성하고 [결정론적 생성으로 이미지 품질 향상](https://huggingface.co/docs/diffusers/main/en/using-diffusers/reusing_seeds) 가이드에 자세히 설명된 대로 시드에서 생성된 이미지를 반복적으로 개선할 수 있습니다.

아래와 같이 시드와 `Generator`를 설정할 수 있습니다. `Generator`로 이미지를 만들면 임의로 새 이미지를 생성하는 대신 매번 동일한 결과가 반환됩니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained(
	"stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16
).to("cuda")
generator = torch.Generator(device="cuda").manual_seed(30)
image = pipeline(
	"Astronaut in a jungle, cold color palette, muted colors, detailed, 8k",
	generator=generator,
).images[0]
image

## 이미지 생성 제어

파이프라인 매개변수 구성 외에도 프롬프트 가중치 및 ControlNet 모델과 같이 이미지 생성 방식을 보다 효과적으로 제어할 수 있는 몇 가지 방법이 있습니다.

### 프롬프트 가중치

프롬프트 가중치는 프롬프트에서 개념의 중요도를 높이거나 낮추어 이미지의 특정 특징을 강조하거나 최소화하는 기술입니다. 가중 프롬프트 임베딩을 생성하는 데 도움이 되는 [Compel](https://github.com/damian0815/compel) 라이브러리를 사용하는 것이 좋습니다.

<Tip>

[프롬프트 가중치](https://huggingface.co/docs/diffusers/main/en/using-diffusers/weighted_prompts) 가이드에서 프롬프트 임베딩을 만드는 방법을 알아보세요. 이 예제에서는 파이프라인에서 프롬프트 임베딩을 사용하는 방법에 중점을 둡니다.

</Tip>

임베딩을 만든 후에는 파이프라인의 `prompt_embeds`(부정 프롬프트를 사용하는 경우 `negative_prompt_embeds`) 매개변수에 전달할 수 있습니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained(
	"stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16
).to("cuda")
image = pipeline(
	prompt_embeds=prompt_embeds, # Compel에서 생성됨
	negative_prompt_embeds=negative_prompt_embeds, # Compel에서 생성됨
).images[0]

### ControlNet

[ControlNet](#controlnet) 섹션에서 보았듯이 이러한 모델은 추가 조건 이미지 입력을 통합하여 이미지를 생성하는 보다 유연하고 정확한 방법을 제공합니다. 각 ControlNet 모델은 특정 유형의 조건 이미지에 대해 사전 학습되어 해당 이미지와 유사한 새 이미지를 생성합니다. 예를 들어 깊이 맵에 대해 사전 학습된 ControlNet 모델을 사용하면 모델에 깊이 맵을 조건 입력으로 제공할 수 있으며 모델은 해당 공간 정보를 보존하는 이미지를 생성합니다. 이는 프롬프트에서 깊이 정보를 지정하는 것보다 빠르고 쉽습니다. [MultiControlNet](https://huggingface.co/docs/diffusers/main/en/using-diffusers/controlnet#multicontrolnet)을 사용하여 여러 조건 입력을 결합할 수도 있습니다!

사용할 수 있는 조건 입력 유형은 다양하며 🤗 Diffusers는 Stable Diffusion 및 SDXL 모델에 대한 ControlNet을 지원합니다. 이러한 모델을 사용하는 방법에 대해 자세히 알아보려면 보다 포괄적인 [ControlNet](https://huggingface.co/docs/diffusers/main/en/using-diffusers/controlnet) 가이드를 참조하세요.

## 최적화

확산 모델은 크기가 크고 이미지를 노이즈 제거하는 반복적인 특성으로 인해 계산 비용이 많이 들고 집약적입니다. 하지만 그렇다고 해서 강력한 GPU 또는 많은 GPU에 액세스해야만 사용할 수 있는 것은 아닙니다. 소비자 및 무료 계층 리소스에서 확산 모델을 실행하기 위한 다양한 최적화 기술이 있습니다. 예를 들어 모델 가중치를 반정밀도로 로드하여 GPU 메모리를 절약하고 속도를 높이거나 전체 모델을 GPU로 오프로드하여 훨씬 더 많은 메모리를 절약할 수 있습니다.

PyTorch 2.0은 PyTorch 2.0을 사용하는 경우 자동으로 활성화되는 [*스케일드 닷 프로덕트 어텐션*](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/fp16#scaled-dot-product-attention)이라는 보다 메모리 효율적인 어텐션 메커니즘도 지원합니다. 이를 [`torch.compile`](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html)과 결합하여 코드를 더욱 빠르게 실행할 수 있습니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16").to("cuda")
pipeline.unet = torch.compile(pipeline.unet, mode="reduce-overhead", fullgraph=True)

메모리를 절약하고 추론 속도를 높이기 위해 코드를 최적화하는 방법에 대한 자세한 내용은 [추론 가속화](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/fp16) 및 [메모리 사용량 줄이기](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/memory) 가이드를 참조하세요.